In [ ]:
#!pip install -e ../../


In [2]:
import time

start_total = time.time()

t0 = time.time()
import logging
print("logging import:", time.time() - t0, "seconds")

t0 = time.time()
from terratorch.tasks import ObjectDetectionTask
print("ObjectDetectionTask import:", time.time() - t0, "seconds")

t0 = time.time()
from terratorch.datamodules.xview import XviewDataModule
print("XviewDataModule import:", time.time() - t0, "seconds")

t0 = time.time()
from lightning.pytorch import Trainer
print("Trainer import:", time.time() - t0, "seconds")

t0 = time.time()
from torchvision import transforms
print("torchvision.transforms import:", time.time() - t0, "seconds")

print("Total import time:", time.time() - start_total, "seconds")

import torch

logging import: 3.24249267578125e-05 seconds


/dccstor/terratorch/users/rkie/gitco/terratorch/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ObjectDetectionTask import: 55.79015898704529 seconds
XviewDataModule import: 0.27486085891723633 seconds
Trainer import: 0.00011301040649414062 seconds
torchvision.transforms import: 7.224082946777344e-05 seconds
Total import time: 56.06584310531616 seconds


In [ ]:
#!wget https://raw.githubusercontent.com/akihironitta/xView-PyTorch/refs/heads/master/datasets.py
#!mv datasets.py ../../terratorch/datasets/xview.py


In [ ]:
img_transform = transforms.Compose([
    transforms.Resize((512, 512)), 
    transforms.ToTensor()
])

dm = XviewDataModule(data_dir = '/dccstor/terratorch/users/rkie/data/xview/train_images', ann_file = '/dccstor/terratorch/users/rkie/data/xview/xView_train.geojson', img_transform = img_transform)

import json
with open('/dccstor/terratorch/users/rkie/data/xview/xView_train.geojson') as f:
    anns = json.load(f)

s = set()
for i, f in enumerate(anns['features']):
    s.add(f['properties']['image_id'])

for setitem in s:
    print(setitem)

In [ ]:
#dm.setup()

#dl = dm.train_dataloader()


In [ ]:
#item = next(iter(dl))

In [ ]:
#item

In [ ]:
#item['boxes_labels']

In [ ]:
#type(images)


In [ ]:
model_args = {
    "framework": "faster-rcnn",
    "backbone": "terramind_v1_tiny",
    "backbone_pretrained": True,
    "num_classes": 100,
    "framework_min_size": 512,
    "framework_max_size": 512,
    "backbone_modalities": ["RGB"],
    "in_channels": 3,
    "necks": [
        {
            "name": "SelectIndices",
            "indices": [2, 5, 8, 11]
        },
        {
            "name": "ReshapeTokensToImage",
            "remove_cls_token": False
        },
        {
            "name": "LearnedInterpolateToPyramidal"
        },
        {
            "name": "FeaturePyramidNetworkNeck"
        }
    ]
}
m = ObjectDetectionTask(model_factory='ObjectDetectionModelFactory', model_args=model_args)

In [ ]:
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint
import torch

checkpoint_cb = ModelCheckpoint(
    dirpath="checkpoints",
    filename="odet-{epoch:02d}-{val_loss:.2f}",
    save_last=True,
)

In [ ]:
trainer = Trainer(
    devices=1,
    precision="16-mixed",
    max_epochs=1,
    default_root_dir='output',
    log_every_n_steps=1,
    check_val_every_n_epoch=1,
    callbacks=[checkpoint_cb]
)

In [ ]:
trainer.fit(model=m, datamodule=dm)  

In [3]:
model = ObjectDetectionTask.load_from_checkpoint("./checkpoints/last.ckpt")

In [9]:
from terratorch.models.object_detection_model_factory import ModelOutputObjectDetection
import torch
from torch import Tensor

class ONNXWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, *args, **kwargs):
        out = self.model(*args, **kwargs)

        # --- Handle ModelOutputObjectDetection ---
        if isinstance(out, ModelOutputObjectDetection):
            if isinstance(out.output, dict):
                # Single image
                boxes = out.output.get("boxes", torch.empty(0, 4))
                scores = out.output.get("scores", torch.empty(0))
                labels = out.output.get("labels", torch.empty(0, dtype=torch.long))
                return boxes, scores, labels

            elif isinstance(out.output, list):
                # Batch: flatten all boxes/scores/labels into single tensors
                all_boxes, all_scores, all_labels = [], [], []
                for item in out.output:
                    all_boxes.append(item.get("boxes", torch.empty(0, 4)))
                    all_scores.append(item.get("scores", torch.empty(0)))
                    all_labels.append(item.get("labels", torch.empty(0, dtype=torch.long)))

                boxes = torch.cat(all_boxes, dim=0) if all_boxes else torch.empty(0, 4)
                scores = torch.cat(all_scores, dim=0) if all_scores else torch.empty(0)
                labels = torch.cat(all_labels, dim=0) if all_labels else torch.empty(0, dtype=torch.long)
                return boxes, scores, labels

        # --- If it's already a tensor ---
        if isinstance(out, Tensor):
            return out

        # --- Fallback ---
        raise TypeError(f"Unsupported model output type: {type(out)}")



In [10]:
onnx_model = ONNXWrapper(model)

In [11]:
onnx_model.eval()


ONNXWrapper(
  (model): ObjectDetectionTask(
    (model): ObjectDetectionModel(
      (torchvision_model): FasterRCNN(
        (transform): GeneralizedRCNNTransform(
            Normalize(mean=[0 0 0], std=[1 1 1])
            Resize(min_size=(512,), max_size=512, mode='bilinear')
        )
        (backbone): BackboneWrapper(
          (backbone): TerraMindViT(
            (encoder_embeddings): ModuleDict(
              (untok_sen2rgb@224): ImageEncoderEmbedding(
                (proj): Linear(in_features=768, out_features=768, bias=False)
              )
            )
            (encoder): ModuleList(
              (0-11): 12 x Block(
                (norm1): LayerNorm()
                (attn): Attention(
                  (qkv): Linear(in_features=768, out_features=2304, bias=False)
                  (attn_drop): Dropout(p=0.0, inplace=False)
                  (proj): Linear(in_features=768, out_features=768, bias=False)
                  (proj_drop): Dropout(p=0.0, inplace=False)


In [12]:

dummy_input = torch.randn(1, 3, 512,512)  # batch size 1, 10 features


In [14]:
torch.onnx.export(
    onnx_model,              # the PyTorch model
    dummy_input,              # dummy input
    "model.onnx",             # output file     
    export_params=True,       # store trained weights
    opset_version=17,         # recommended ONNX opset
    do_constant_folding=True, # optimize constants
    input_names=['input'],    # model input name
    output_names=['output'],  # model output name
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}  # allow variable batch size
)

------------------
BackboneWrapper(
  (backbone): TerraMindViT(
    (encoder_embeddings): ModuleDict(
      (untok_sen2rgb@224): ImageEncoderEmbedding(
        (proj): Linear(in_features=768, out_features=768, bias=False)
      )
    )
    (encoder): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm()
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=False)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm()
        (mlp): GatedMlp(
          (fc1): Linear(in_features=768, out_features=2048, bias=False)
          (act): SiLU()
          (fc2): Linear(in_features=2048, out_features=768, bias=False)
          (fc3): Linear(in_features=768, out_features=2048, bias=False)
        )
      )
    )
    (encoder_norm): LayerNorm()
    (tokenize

In [ ]:
print(sample[0])  


In [ ]:
print(sample[1])  


In [18]:
import onnx
import onnxruntime as ort
import numpy as np

# Path to your ONNX model
onnx_path = "model.onnx"

# --- Load the model to inspect input/output metadata ---
model = onnx.load(onnx_path)
onnx.checker.check_model(model)

# Create inference session
session = ort.InferenceSession(onnx_path)

# Get input metadata
input_meta = session.get_inputs()[0]
input_name = input_meta.name
input_shape = input_meta.shape  # may have symbolic dimensions

# Replace symbolic dimensions with actual values where possible
resolved_shape = [
    dim if isinstance(dim, int) else 1 for dim in input_shape
]

# If the model expects fixed spatial dimensions, use them
# Example: input_shape could be ['batch_size', 3, 512, 512]
height = resolved_shape[2]
width = resolved_shape[3]

print(f"Input name: {input_name}")
print(f"Expected input shape: {input_shape} → resolved: {resolved_shape}")

# Create a dummy input with the correct shape
dummy_input = np.random.randn(
    resolved_shape[0],  # batch
    resolved_shape[1],  # channels
    height,
    width
).astype(np.float32)

# Run inference
outputs = session.run(None, {input_name: dummy_input})

# Print output names and shapes
for name, arr in zip([o.name for o in session.get_outputs()], outputs):
    print(f"Runtime output '{name}': shape={arr.shape}, dtype={arr.dtype}")


Input name: input
Expected input shape: ['batch_size', 3, 512, 512] → resolved: [1, 3, 512, 512]
Runtime output 'output': shape=(0, 4), dtype=float32
Runtime output '3185': shape=(0,), dtype=float32
Runtime output '3186': shape=(0,), dtype=int64
